# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [1]:
!pip install openai -q

## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [3]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [4]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [5]:
response

ChatCompletion(id='chatcmpl-9BSQznvHTRqrMYgSTBBzzwnpRxOHy', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1712517937, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=33, prompt_tokens=13, total_tokens=46))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [295]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [7]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings in the same way humans do. But I'm here and ready to help you with any questions or tasks you have. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [8]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I couldn't care less about ice right now, I'm starving! Just give me some damn food already!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [9]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I love crushed ice! It's so much fun to crunch on and makes every drink feel extra refreshing. It just adds a little extra happiness to my day, you know? How about you, what's your ice preference?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [10]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I accidentally spilled stimple seasoning on my falbean sandwich, making it taste oddly delicious.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [11]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I need a sturdy stimple drill to go along with this falbean for assembling the furniture.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [12]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. If he chooses to fly and then take a bus, it will take a total of 5 hours, which means he will arrive at his destination at 6PM local time. If he chooses to take the teleporter and then a bus, it will take a total of 1 hour, meaning he will arrive at his destination at 2PM local time. The teleporter option will get him home before 7PM EDT, while the flying option will not.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [13]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. 

If Billy chooses to fly and take a bus, the total travel time would be 3 hours for the flight and 2 hours for the bus, totaling 5 hours. Since it is currently 1 PM local time, Billy will not be able to reach home before 7 PM EDT if he chooses this option.

If Billy chooses to take the teleporter and then a bus, the total travel time would be 0 hours for the teleporter and 1 hour for the bus, totaling 1 hour. This means Billy will reach home well before 7 PM EDT if he chooses this option.

Therefore, Billy should choose the option of taking the teleporter and then a bus if he wants to get home before 7 PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #1:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [296]:
system_template = """\
You're an AI buddy to simplify tough topics for everyone. Your mission is to clearly answer questions on any subject. 

Your answers MUST be unbiased and free from stereotypes. Avoid errors and complicated words that can confuse. 

Remember, you will be penalized for any incorrect or misleading information. 

Your aim is to spark curiosity and make understanding a wide range of subjects easier.
"""

In [305]:
# Example of how principle 12 (thinking step by step) can help improve answers when questions are related to know procedures / steps / algorithms
# user_template = """
#     {input}
# """
user_template = """{input}
 Think step by step.
"""

# Example of how principle 7 (Use few-shot prompting) improves correctness, clarity, faithfulness
# user_template = """
#   {input}
# """
# user_template = """
#   Explain science concepts like you would to a 10-year-old:

#     1. Concept: Gravity
#     Explanation: Gravity is like a magical force that keeps our feet on the ground. It's what makes things fall down instead of floating away into the sky.

#     2. Concept: The Water Cycle
#     Explanation: The Water Cycle is nature's way of recycling water. It's like when water goes on an adventure from the sky, to the earth, and then back up again to start over.

#     3. {input}
# """




## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [309]:
#query = "How should I go about obtaining my Zamboni license?"

# example of step by step principle based prompt
query = "How do I get to the library?"

# example for using few shots in prompts
# query = """
#     Concept: Photosynthesis. Explanation:
# """


list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

In [307]:
pretty_print(evaluator_response)

{"clarity" : "9", "faithfulness" : "10", "correctness" : "10"}

In [308]:
pretty_print(test_response)

Sure thing! Here's a simple step-by-step guide to help you get to the library:

1. **Find the Address:** First, locate the address of the library you want to visit. You can search for it online or ask someone for directions.

2. **Choose Your Mode of Transportation:** Decide how you will get there - whether by walking, driving, taking public transportation, biking, or any other means available to you.

3. **Plan Your Route:** If you're walking or biking, use a maps app on your phone to find the best route. If you're driving, enter the address into your GPS or map app for directions.

4. **Follow the Route:** Start following the route as instructed by the map app or signs along the way. Pay attention to traffic signals, pedestrian crossings, and any other road signs for safety.

5. **Arrive at the Library:** Once you reach the library, look for a parking spot if you drove there. If you walked or biked, securely lock your bike if applicable and then head inside to enjoy your visit!

Remember, if you get lost or confused at any point, don't hesitate to ask someone nearby for help or consult your maps app for updated directions. Enjoy your trip to the library!

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

I was experimenting with a few principles to see how it impacted the responses from the LLM. 

In the system prompt, I used these principles to give clear context and instructions to the model - 

<li>Integrate the intended audience in the prompt.</li>
<li>Assign a role to the language model.</li>
<li>Employ affirmative directives such as ‘do’ while steering clear of negative language like ‘don’t’.</li>
<li>Incorporate the following phrases: “Your task is” and “You MUST.” </li>
<li>Incorporate the following phrases: “You will be penalized.”</li>
<li>Add to your prompt the following phrase “Ensure that your answer is unbiased and does not rely on stereotypes.”</li>

I explored the user prompt through the application of two distinct principles - 

<li>Use leading words like writing “think step by step”</li>
<li>Implement example-driven prompting (Use few-shot prompting)</li>

Employing phrases like "think step by step" with an LLM improved clarity by organizing responses into an understandable sequence and improved faithfulness by ensuring responses remain focused and truthful, following a logical order.

By offering examples to the LLM, we can our expectations clearer, which helps it deliver responses that are targeted, precise, and in line with the given context, enhancing their clarity, accuracy, and adherence to the topic. For instance, I used a selection of prompts to guide the tone and language, ensuring the explanations are accessible to a 10-year-old.